# 50.039 Theory and Practice of Deep Learning Project 2024

Group 10
- Issac Jose Ignatius (1004999)
- Mahima Sharma (1006106)
- Dian Maisara (1006377)


## Motivation

Chest radiography is an essential diagnostic tool used in medical imaging to visualise structures and organs within the chest cavity. It is crucial for diagnosing various respiratory and heart-related conditions. However, with the increased demand for radiological reports within shorter timeframes to detect and treat illnesses, there have been insufficient radiologists available to perform such tasks at scale. Therefore, automated chest radiograph interpretation could provide substantial benefits supporting large-scale screening and population health initiatives. Deep-learning algorithms can be used to bridge this gap. They have been used for image classification, anomaly detection, organ segmentation, and disease progression prediction.

*In this project, we aim to train a deep neural network to perform multi-label image classification on a wide array of chest radiograph images that exhibit various pathologies.*<br><br>

---




## Import all relevant libraries

In [ ]:
%pip install --upgrade torchmetrics
%pip install --upgrade tqdm

In [ ]:
# Numpy
import numpy as np
# Pandas
import pandas as pd

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler
import torchvision
print(torchvision.__version__)
import torchvision.transforms as T
from torchvision.transforms import v2
import torchvision.transforms.functional as fn
from torchvision.io import read_image, ImageReadMode

# Image
# import PIL
# from PIL import ImageFile

# File Operations
from glob import glob
import os

# Helper scripts
# import sys
# sys.path.insert(0, '../src')
# from saver_loader import *
# %reload_ext autoreload
# %autoreload 2

In [ ]:
import gc

torch.cuda.empty_cache()
gc.collect()

In [ ]:
# Use GPU if available, else use CPU
device = torch.device("cuda") # if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu") 
print(device)

## Data Loading

The training and validation datasets are from the **CheXphoto dataset** (Philips et al., 2020). <br><br> CheXphoto comprises a training set of natural photos and synthetic transformations of 10,507 X-ray images from 3,000 unique patients (32,521 data points) sampled at random from the CheXpert training dataset and an accompanying validation set of natural and synthetic transformations applied to all 234 X-ray images from 200 patients with an additional 200 cell phone photos of x-ray films from another 200 unique patients (952 data points).

## Setup environmental variables

In [ ]:
data_path = os.path.join(os.path.abspath(''), "../ChexPhoto/chexphoto-v1")
print(data_path)

### Loading dataset (image and labels)

In [ ]:
# Bug in Path present in training dataset
def fix_error_paths(row):
    row = row.replace("//", "/")
    return row

def str_to_array(row):
    row = row.replace("[", "").replace("]", "").replace(" ", "").split(".")[:-1]
    return np.array(row).astype(np.float32)

In [ ]:
labels = ['Pleural Effusion', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Edema', 'Pleural Other', 'Fracture', 'Support Devices']

train_df = pd.read_csv("../data/processed/train_one_hot_encoded.csv", index_col=False)
train_df["Path"] = train_df["Path"].apply(fix_error_paths)
valid_df = pd.read_csv("../data/processed/valid_one_hot_encoded.csv", index_col=False)
#test_df = pd.read_csv("../data/processed/test_one_hot_encoded.csv", index_col=False)

for label in labels:
    train_df[label] = train_df[label].apply(str_to_array)
    valid_df[label] = valid_df[label].apply(str_to_array)
    #test_df[label] = test_df[label].apply(str_to_array)

### Visualize dataframes

In [ ]:
display(train_df)
display(valid_df)
#display(test_df)

### Custom Dataset Implementation

In [ ]:
# ImageFile.LOAD_TRUNCATED_IMAGES = True
# torchvision.set_image_backend('PIL')

# Implementation of Custom Dataset Class for CheXPhoto Dataset
class CheXDataset(torch.utils.data.Dataset):
    # Accepts dataframe object and str
    def __init__(self, df: pd.DataFrame): #, split: str):
        self.dataframe = df
        # Note: self.split is used to identify test set in __getitem__ method due to the arrangement of values in tensor
        #self.split = split

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x_path = data_path + "/" + self.dataframe.iloc[idx, 0].split("CheXphoto-v1.0", 1)[-1]
        transform = T.Compose([
            v2.Resize((512, 512), interpolation=T.InterpolationMode.BICUBIC)#,
            #T.ToTensor()
        ])
        #resized_x_tensor = transform(PIL.Image.open(x_path).convert('RGB'))
        resized_x_tensor = transform(read_image(x_path, mode = ImageReadMode.RGB)) /255

        # Perform different slicing operations based on the split - focus is on handling test split
        # Additional input features e.g. sex, age, FoL, AoP are removed for now
        # if self.split.lower() == "test":
        y_str = self.dataframe.iloc[idx, 1:].to_string(index=False)
        y_arr = np.array(y_str.replace("[","").replace("]","").split(","), dtype=np.float32)
        y = torch.from_numpy(y_arr)
        # else:
        #   y = torch.tensor(self.dataframe.iloc[idx, 5:], dtype=torch.float32)

        return resized_x_tensor, y

### Subsetting dataset (Binary Classification for Pleural Effusion)

In [ ]:
def preprocess_pEffClassification(df):
    df.drop(columns=['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Edema', 'Pleural Other', 'Fracture', 'Support Devices'], axis=1, inplace=True)
    return df

# # Drop all other labels
pEff_train = preprocess_pEffClassification(train_df)
pEff_valid = preprocess_pEffClassification(valid_df)
#pEff_test = preprocess_pEffClassification(test_df)

display(pEff_train)
display(pEff_valid)
#display(pEff_test)

### Custom Dataloader

In [ ]:
# Load into custom Dataset
pEff_train_data = CheXDataset(pEff_train)
pEff_valid_data = CheXDataset(pEff_valid)
#pEff_test_data = CheXDataset(pEff_test, "test")

# Prepare random sampler for training subset (10% of samples)
# pEff_train_sampler = RandomSampler(pEff_train_data, replacement=True, num_samples=int(1*len(edema_train_data))) # 0.1

In [ ]:
# Load into DataLoader
batch_size = 32
pEff_train_loader = DataLoader(pEff_train_data, batch_size, shuffle=True) #, sampler=edema_train_sampler), pin_memory=True
pEff_valid_loader = DataLoader(pEff_valid_data, batch_size)
#pEff_test_loader = DataLoader(pEff_test_data, batch_size)

In [ ]:
x, y = pEff_train_data[0]
print(x, x.shape, x.dtype)
print(y, y.shape, y.dtype)

## Model Tuning

Our initial model is a simple feedforward neural network with multiple heads (14 heads) capable of classifying each observation for the various pathologies. We utilise the Cross-entropy loss function to optimise the model during training.

**This is a TODO since it can change**


### First Iteration - Convolutional Neural Network

In [ ]:
# We will be using Convolutional Neural Network
class FullCNN(nn.Module):
    def __init__(self):
        super(FullCNN, self).__init__()

        # Conv Layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1)

        # BN Layers
        self.batch_norm1 = nn.BatchNorm2d(16)
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.batch_norm3 = nn.BatchNorm2d(64)
        self.batch_norm4 = nn.BatchNorm2d(128)

        # MP Layer
        self.maxpool2d = F.max_pool2d

        # FC Layers
        self.fc1 = nn.Linear(128*32*32, 8192, dtype = torch.float32)
        self.fc2 = nn.Linear(8192, 512, dtype = torch.float32)
        self.fc3 = nn.Linear(512, 32, dtype = torch.float32)
        self.fc4 = nn.Linear(32, 3, dtype = torch.float32)

        # DO Layers
        self.dropout = nn.Dropout(0.2)

        # TODO: Skip connection implementation
        # Skip connection implementation
        # self.skip_connection1 = nn.Conv2d(16, 64, kernel_size=1, stride=1)
        # self.skip_connection2 = nn.Conv2d(32, 128, kernel_size=1, stride=1)

        # Softmax Layer - Not needed when using CrossEntropyLoss
        # self.softmax = F.softmax


    def forward(self,x):
        # All forward operations
        x1 = x.to(torch.float32)
        # First convolutional layer
        x1 = self.conv1(x1)
        x1 = self.batch_norm1(x1)
        x1 = F.relu(x1)
        x1 = self.maxpool2d(x1, 2)
        
        # Second convolutional layer -> pooling
        x2 = self.conv2(x1)
        x2 = self.batch_norm2(x2)
        x2 = F.relu(x2)
        x2 = self.maxpool2d(x2, 2)

        # Apply skip connection after the second convolutional layer
        # skip_connection_output1 = self.skip_connection1(x1)
        # x2 += skip_connection_output1

        # Third convolutional layer -> pooling
        x3 = self.conv3(x2)
        x3 = self.batch_norm3(x3)
        x3 = F.relu(x3)
        x3 = self.maxpool2d(x3, 2)

        # Apply skip connection after the third convolutional layer
        # skip_connection_output2 = self.skip_connection2(x2)
        # x3 += skip_connection_output2

        # Fourth convolutional layer -> pooling
        x4 = self.conv4(x3)
        x4 = self.batch_norm4(x4)
        x4 = F.relu(x4)
        x4 = self.maxpool2d(x4, 2)

        # print(x.shape)
        # Flatten output of convolutions
        x4 = x4.view(-1, 128*32*32)
        # print(x.shape)
        x4 = self.dropout(x4)

        # First FC layer
        x5 = self.fc1(x4)
        x5 = F.relu(x5)
        x5 = self.dropout(x5)

        # Second FC layer
        x6 = self.fc2(x5)
        x6 = F.relu(x6)
        x6 = self.dropout(x6)

        # Third FC layer
        x7 = self.fc3(x6)
        x7 = F.relu(x7)
        x7 = self.dropout(x7)
        
        # Fourth FC layer
        x8 = self.fc4(x7)
        #x = self.softmax(x) # No need, Refer to https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss

        return x8

In [ ]:
# Create Model
#model = torchvision.models.resnet18(weights=False).to(device)
model = FullCNN().to(device)
print(model)

#### Training CNN

In [ ]:
from tqdm.notebook import tnrange, tqdm
# torch.cuda.empty_cache()
# del model
# import gc
# gc.collect()
def train(model, train_loader, valid_loader, epochs=10, lr=1e-3):
    # Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss().to(device)
    # print(accuracy.device)  
    #train_loss_values = []
    #train_accuracy_values = []
    for epoch in tnrange(epochs):
        # print(epoch)
        model.train()

        train_loss = 0.0
        train_total = 0
        train_correct = 0

        for inputs, outputs in tqdm(train_loader):
            inputs_re, outputs_re = inputs.to(device), outputs.to(device)
            optimizer.zero_grad()
            preds = model(inputs_re)
            #print([torch.argmax(outputs_re[i], dim=0, keepdim=True) for i in range(batch_size)]) DONT DELETE
            #print(torch.stack([torch.argmax(outputs_re[i], dim=0, keepdim=True) for i in range(batch_size)],dim=0)) DONT DELETE
            #resized_out = torch.tensor([torch.argmax(outputs_re[i], dim=0, keepdim=True) for i in range(batch_size)]).to(device)
            # print("1:",torch.argmax(outputs_re, dim=1))
            # print("2:",(torch.argmax(outputs_re, dim=1).reshape(-1,1)))
            # print("3:",torch.argmax(outputs_re, dim=1).unsqueeze(1))

            loss_value = loss(preds, torch.argmax(outputs_re, dim=1))# Cross Entropy accepts class labels so we need to convert the OHE
            loss_value.backward()
            optimizer.step()

            # Compute metric
            train_loss += loss_value.item()
            train_total += outputs_re.size(0)
            #print("Preds argmax", torch.argmax(preds, dim=1))
            #print("Outputs_re original argmax", torch.argmax(outputs_re, dim=1))
            train_correct += (torch.argmax(preds, dim=1) == torch.argmax(outputs_re, dim=1)).sum().item() # Convert both to class labels
            
        # Rewrite implementation for evaluation. Note: Dropout and BatchNorm layers are active when model is in train mode. 
        # Need to disable them before computing losses and accuracy with model.eval()
        train_loss /= len(train_loader)
        train_accuracy = train_correct / train_total

        model.eval() # handles Dropout (turn off) and BatchNorm (running stats) layers automatically
        
        val_total = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, outputs in tqdm(valid_loader):
                # Retrieve predictions
                inputs_re, outputs_re = inputs.to(device), outputs.to(device)
                preds = model(inputs_re)

                # Change the line to as follows -
                #final_preds = torch.where(preds < 0.5, 0, 1)
                #final_preds = final_preds.to(device)
                # print(torch.argmax(preds, dim=1))
                # print(torch.argmax(outputs_re, dim=1))
                # print("\n")

                # Compute metrics
                val_total += outputs_re.size(0)
                val_correct += (torch.argmax(preds, dim=1) == torch.argmax(outputs_re, dim=1)).sum().item() # Convert both to class labels

        val_accuracy = val_correct / val_total    
        print(f'--- Epoch {epoch+1}/{epochs}: Train loss: {train_loss:.4f}, Train accuracy: {train_accuracy:.4f}\n Validation accuracy: {val_accuracy}')


In [ ]:
train(model, pEff_train_loader, pEff_valid_loader)
#torch.save(model.state_dict(), 'weights.pt')

## Observations

**TODO** Discuss whether its right for us to pluck all our evaluation and training together and discuss it here or break up the code without any descriptions


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d056a7b8-1929-4f43-a228-a643b0e765c5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>